## This tutorial presents how to simulate the full network with time dependent dynamic input currents into arbitrary set of neurons. If you haven't completed the tutorial 1, it is recommended to do so.

## Step 1: Import necessary packages and modules

In [ ]:
# This line allows you to display matplotlib plots within the Jupyter Notebook
%matplotlib inline

# Import External packages 

import os
import numpy as np
import matplotlib.pyplot as plt

# Configure the working directory (Important: This should be set to home directory of 'dynworm' folder)

default_dir = os.path.dirname(os.getcwd())
os.chdir(default_dir)

# Import Main module

import dynworm as dw

## Step 1: Initialize neural parameters and connectivity

In [ ]:
dw.network_sim.initialize_params_neural()
dw.network_sim.initialize_connectivity()

## Step 2: Define dynamic input stimuli
### The dynamic stimuli should be in the form of numpy.array of shape (timesteps x network size), which define the time dependent inputs into all neurons
### In this tutorial, we will use in-built functions from dynworm.utils which can set up sinusoidal, noisy and step inputs
### You can also construct your own dynamic stimuli with custom functions
### Identical to constant input, dynamic input is defined with normalized stimuli with following conversion: 0 - 1 (normalized) <--> 0 - 10nA (nano-ampere)
### IMPORTANT: As of now, all dynamic stimuli matrices should have row numbers (i.e. timesteps) 1 second longer than the actual ending time of your simulation. e.g. if you want to simulate 20s with the time resolution of 0.01s, your dynamic stimuli matrix should have dimension (2100, 279) with extra 100 input data points past 2000 (20s)
### This is to avoid the out of range interpolation error of input during network integration
### Also, try to avoid a time interval with zero input (i.e. no input to any neuron) to the network. This often results in an interpolation error as well (This issue will be resolved in next alpha version) 

In [ ]:
# neurons_idx module has dictionary 'neurons_list' where you can easily reference the information about neurons of interest
# degree: total number of synaptic inputs 
# group: group name of neuron

neurons_list = dw.neurons_idx.neurons_list
neurons_list[276]

### In this tutorial we define each of sinusoidal and noisy input into PLM neurons using the ce_utils.construct_dyn_inputmat function

### The function accepts:
### t0 - Starting time point
### tf - Ending time point 
### dt - Time resolution
### input_type - 'sinusoidal' or 'noisy'
### neuron_indices - List of neurons' indices to be stimulated
### normalized_amps - with input_type = 'sinusoidal', list of amplitudes of sine waves. With input_type = 'noisy', list of base input amplitudes without noises
### freqs - Exclusive to input_type = 'sinusoidal'. Determines frequencies of sine wave inputs
### noise_amplitude - Exclusive to input_type = 'noisy'. Determines the noise amplitudes of noisy inputs
### Note that normalized_amps, freqs and noise_amplitude should follow the same order of neurons as defined in neuron_indices

In [ ]:
input_mat_sinusoidal = dw.utils.construct_dyn_inputmat(t0=0, tf=21, input_type='sinusoidal', \
                                                       neuron_indices=[276, 278], normalized_amps=[0.2, 0.3], \
                                                       freqs=[3, 4])

In [ ]:
input_mat_noisy = dw.utils.construct_dyn_inputmat(t0=0, tf=21, input_type='noisy', \
                                                  neuron_indices=[276, 278], normalized_amps=[0.2, 0.3], 
                                                  noise_amplitudes=[0.05, 0.05])

### The figure below shows the different dynamic input into PLMR

In [ ]:
fig = plt.figure(figsize=(9, 12))

plt.subplot(211)
plt.title('Sinusoidal input')
plt.plot(input_mat_sinusoidal[:, 276], color = 'black')

plt.subplot(212)
plt.title('Noisy input')
plt.plot(input_mat_noisy[:, 276], color = 'black')

## Step 3: Configure ablation mask array
### Ablation mask is initially all set to True (i.e. all neurons are present in the network)

In [ ]:
ablation_mask = np.ones(dw.network_sim.params_obj_neural['N'], dtype = 'bool')

## Step 4: Run the simulation
### run_network_dyninput in ce_network module simulates the network when given with input_mat array
### t_delta: time resolution of simulation. This is not to be confused with integration step, which is determined internally
### input_mat: the input_mat array with configured time dependent inputs of dimension (timepoints x 279)
### ablation_mask: the ablation vector which define any neurons to be ablated

In [ ]:
result_dict_sinusoidal = dw.network_sim.run_network_dyninput(input_mat=input_mat_sinusoidal, 
                                                  ablation_mask = ablation_mask)

In [ ]:
result_dict_noisy = dw.network_sim.run_network_dyninput(input_mat=input_mat_noisy, 
                                                  ablation_mask = ablation_mask)

## Step 5: Extract the result
### run_network_constinput outputs a dictionary object with following keys
### 'v_solution' : The membrane voltage activities of all neurons in respect to their resting potential (dimension: timepoints x 279)
### 'steps' : number of solution points between t_start and t_final
### 't' : time vector of the simulation 
### 'trajectory_mat': The membrane voltage activities of all neurons prior being subtracted by resting potential (dimension: timepoints x 279)
### 'v_threshold': The resting potentials of all neurons given the constant input profile 

In [ ]:
result_dict_sinusoidal.keys()

### In this example, we will take a look at the 'v_solution' 
### We take transpose to change the shape to spatial neurons x temporal axis 

In [ ]:
v_sol = result_dict_sinusoidal['v_solution'].T

### One can use matplotlib.pyplot.pcolor to visualize the activities of all neurons 

In [ ]:
fig = plt.figure(figsize=(15, 8))
plt.pcolor(v_sol[:, 100:], cmap='bwr', vmin = -20, vmax = 20)
plt.colorbar()

### Or only the motorneuronal activities

In [ ]:
fig = plt.figure(figsize=(15, 8))
plt.pcolor(v_sol[dw.neurons_idx.motor_group, 100:-100], cmap='bwr')
plt.colorbar()

### Here we show the activities of Ventral Type B, D and Dorsal Type B, D neurons

In [ ]:
fig = plt.figure(figsize=(12, 8))
plt.subplot(4,1,1)
plt.pcolor(v_sol[dw.neurons_idx.VB_ind, 100:600], cmap='bwr')
plt.ylim(len(dw.neurons_idx.VB_ind), 0)
plt.colorbar()

fig = plt.figure(figsize=(12, 8))
plt.subplot(4,1,2)
plt.pcolor(v_sol[dw.neurons_idx.VD_ind, 100:600], cmap='bwr')
plt.ylim(len(dw.neurons_idx.VD_ind), 0)
plt.colorbar()

fig = plt.figure(figsize=(12, 8))
plt.subplot(4,1,3)
plt.pcolor(v_sol[dw.neurons_idx.DB_ind, 100:600], cmap='bwr')
plt.ylim(len(dw.neurons_idx.DB_ind), 0)
plt.colorbar()

fig = plt.figure(figsize=(12, 8))
plt.subplot(4,1,4)
plt.pcolor(v_sol[dw.neurons_idx.DD_ind, 100:600], cmap='bwr')
plt.ylim(len(dw.neurons_idx.DD_ind), 0)
plt.colorbar()

## Additionally, ce_plotting module provides useful plotting functions such as low dimensional projection

In [ ]:
dw.plotting.plot_DominantModes(result_dict = result_dict_sinusoidal, sub_neurons = dw.neurons_idx.motor_group)